In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2024-07-17 15:45:56.029037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 15:45:56.029150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 15:45:56.197400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import torch

In [4]:
import pandas as pd

In [5]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [6]:
HF_TOKEN = 'your hf token'

In [7]:
df = pd.read_csv('/kaggle/input/train-csv/train_data.csv')

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# Update 'id' column
df['id'] = df.apply(lambda row: row['Technique'] if row['ID'] == "No sub-Techniques" else row['ID'], axis=1)

In [10]:
# Count the number of occurrences for each tactic name
tactic_counts = df['Tactic-Name'].value_counts()

In [11]:
# Filter tactic names with less than 100 occurrences
filtered_tactics = tactic_counts[tactic_counts < 100].index
df_filtered = df[df['Tactic-Name'].isin(filtered_tactics)]

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [13]:
# Create a new list for results
result = []
pad_token_id = tokenizer.eos_token_id

In [14]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
torch.cuda.empty_cache()

In [15]:
a=0
# Iterate over the filtered DataFrame and generate descriptions
for index, row in df_filtered[268:].iterrows():
    print(a)
    a+=1
    technique_id = row['id']
    tactic_name = row['Tactic-Name']
    technique_name = row['Technique-Name']
    description = row['Description']
    
    prompt = (
        f"Generate a paragraph of 5 threat-action sentences for the technique with ID = {technique_id}, "
        f"tactic name = {tactic_name}, and technique name = {technique_name}, and Description = {description}. "
        "Ensure that the sentences are diverse and do not replicate descriptions from the MITRE ATT&CK framework."
    )
    
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert tasked with generating unique and varied technique descriptions."},
        {"role": "user", "content": prompt}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    outputs = model.generate(
        input_ids,
        max_new_tokens=2056,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]
    generated_text = tokenizer.decode(response, skip_special_tokens=True)
    
    # Add the generated text to the new list
    result.append({
        "id": technique_id,
        "tactic_name": tactic_name,
        "technique_name": technique_name,
        "description": description,
        "generated_sentences": generated_text
    })

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


199


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


200


KeyboardInterrupt: 

In [16]:
# Create a DataFrame from the results and save to CSV
results_df = pd.DataFrame(result)

In [17]:
results_df.to_csv('generated_threat_action_sentences.csv', index=False)